<a href="https://colab.research.google.com/github/yael-vinker/CLIPSketch/blob/main/CLIPSketch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *CLIPSketch* - sketch your own image

## Install Dependencies and Clone the Repo

Make sure your Hardware accelerator is set to GPU.

Runtime > Change runtime type > Hardware Accelerator 

In [4]:
# !git clone https://github.com/yael-vinker/CLIPSketch.git

import os
from getpass import getpass
user = getpass('GitHub user')
password = getpass('GitHub password')
os.environ['GITHUB_AUTH'] = user + ':' + password

!git clone https://$GITHUB_AUTH@github.com/yael-vinker/CLIPSketch.git

GitHub user··········
GitHub password··········
fatal: destination path 'CLIPSketch' already exists and is not an empty directory.


In [ ]:
%cd CLIPSketch
!pip install -r requirements.txt
!pip install git+https://github.com/openai/CLIP.git
!git clone https://github.com/BachiLi/diffvg
%cd diffvg
!git submodule update --init --recursive
!python setup.py install

In [2]:
%cd /content/CLIPSketch

import subprocess as sp
import numpy as np
import os
import multiprocessing as mp
import warnings
import pydiffvg
import torch

from torch.nn.parallel import parallel_apply
from shutil import copyfile
from PIL import Image

#Make sure your Hardware accelerator is set to GPU.
#Runtime > Change runtime type > Hardware Accelerator
print(torch.cuda.is_available())
device = torch.device("cuda" if (torch.cuda.is_available() and torch.cuda.device_count() > 0) else "cpu")
print(device)
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
warnings.filterwarnings(action='once')

manager = mp.Manager()
exit_codes = []
losses_all = manager.dict()

if not os.path.isfile("/content/CLIPSketch/U2Net_/saved_models/u2net.pth"):
    sp.run(["gdown", "https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ", "-O", "U2Net_/saved_models/"])

test_name = "camel"
output_dir = f"/content/CLIPSketch/output_sketches/{test_name}/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

/content/CLIPSketch
True
cuda


In [17]:

def run(target, seed, wandb_name):
    proc = sp.Popen(["python", "-W","ignore", "painterly_rendering.py", target, 
                                "--output_dir", output_dir,
                                "--wandb_name", wandb_name,
                                "--num_iter", str(num_iter),
                                "--save_interval", str(save_interval),
                                "--seed", str(seed),
                                "--use_gpu", str(use_gpu),
                                "--fix_scale", str(fix_scale), 
                                "--mask_object", str(mask_object),
                                "--mask_object_attention", str(mask_object_attention)])
    try:
      outs, errs = proc.communicate(timeout=20000)
    except Exception as e:
      logging.error(traceback.format_exc())
    config = np.load(f"{output_dir}/{wandb_name}/config.npy", allow_pickle=True)[()]
    loss_eval = np.array(config['loss_eval'])
    inds = np.argsort(loss_eval)
    losses_all[wandb_name] = loss_eval[inds][0]

In [18]:
%cd /content/CLIPSketch/
import traceback
import logging
targets = ["/content/CLIPSketch/target_images/camel.png"]
target_names = ["camel"]
num_iter = 20
save_interval = 5
use_gpu=1
seeds = [100]

# if you need to mask the input image and pad the aspect ratio
fix_scale=0
mask_object=0
mask_object_attention=0

ncpus=10
P = mp.Pool(ncpus) # Generate pool of workers
for target, target_name in zip(targets, target_names): # Generate processes
    for seed in seeds:
        wandb_name=f"{target_name}_seed{seed}"
        # p = sp.Popen('/bin/bash')
        # !python -W ignore painterly_rendering.py \
        # target
        P.apply_async(run,(target, seed, wandb_name)) # run simulation and ISF analysis in each process
        # run(target, seed, wandb_name)

    P.close()
    P.join() # start processes 

    sorted_final = dict(sorted(losses_all.items(), key=lambda item: item[1]))
    copyfile(f"{output_dir}/{list(sorted_final.keys())[0]}/best_iter.svg", f"{output_dir}/{list(sorted_final.keys())[0]}_best.svg")


/content/CLIPSketch


Lossy conversion from float32 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
[W accumulate_grad.h:170] Warning: grad and param do not obey the gradient layout contract. This is not an error, but may impair performance.
grad.sizes() = [2048, 512, 1, 1], strides() = [512, 1, 1, 1]
param.sizes() = [2048, 512, 1, 1], strides() = [512, 1, 512, 512] (function operator())


results will be saved to /content/CLIPSketch/output_sketches/camel/camel_seed100
cuda
test epoch[0/20] loss[0.317962646484375] time[0.37354564666748047]
test epoch[10/20] loss[0.30161285400390625] time[0.26520490646362305]


In [11]:
%cd /content/CLIPSketch/
!python general_test.py

/content/CLIPSketch
in main
results will be saved to output_sketches/camel/camel_seed0
cuda
{'target': 'target_images/camel.png', 'output_dir': 'output_sketches/camel/camel_seed0', 'path_svg': 'none', 'use_gpu': 1, 'seed': 0, 'mask_object': 0, 'fix_scale': 0, 'use_wandb': 0, 'wandb_user': 'yael-vinker', 'wandb_name': 'camel_seed0', 'wandb_project_name': 'none', 'num_iter': 2, 'num_stages': 1, 'lr_scheduler': 0, 'lr': 1.0, 'color_lr': 0.01, 'color_vars_threshold': 0.0, 'batch_size': 1, 'save_interval': 1, 'eval_interval': 10, 'image_scale': 224, 'num_paths': 16, 'width': 1.5, 'control_points_per_seg': 4, 'num_segments': 1, 'attention_init': 1, 'saliency_model': 'clip', 'saliency_clip_model': 'ViT-B/32', 'xdog_intersec': 1, 'mask_object_attention': 0, 'softmax_temp': 0.3, 'percep_loss': 'none', 'perceptual_weight': 0, 'train_with_clip': 0, 'clip_weight': 0, 'start_clip': 0, 'num_aug_clip': 4, 'include_target_in_aug': 0, 'augment_both': 1, 'augemntations': 'affine', 'noise_thresh': 0.5, '